# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 48](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=48)*

If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the URL you used to get the error. 

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-11-09"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "true"  # TODO change to false before push, else true

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

verbose = (verbose == "true") 


In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241109&max_day_obs=20241110
DBG get_records len(page)=3 len(recs)=0
DBG get_records-2 len(page)=3 len(recs)=3
DBG get_records-2 len(self.records)=3 status={'endpoint_url': 'https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241109&max_day_obs=20241110', 'number_of_records': 3, 'error': None}
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
DBG get_exposures len(page)=0 len(recs)=0
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LATISS&order_by=

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-09 to 2024-11-10 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: Simonyi

```
We started around 12:00h CLT powering up the rotator and hexapods. Then we did the M1M3 HP breakaway test passing all HPs. We ran this from EUI to run all HPs at the same time. We ran bump test with the script, and 5 FAs failed (215, 227, 235, 409's primaries and 330 secondary). M1M3 faulted when we put it to disabled state to switch to EUI mode to run the individual FA bump test, it is known issue. OBS-652. We waited air pressure to be lowered to Enable back. Meanwhile we ran hexapod warmups, and comcam usdf checkout. 
We re-ran the individual bump test, FA 330 passed, 227, 235, 409 seemed to be within the range, but Petr identified FA 215 as bad and he asked to close its valve from the M1M3 cell, Brian helped us to performed this action in the cell.
During the hexapod warmups, M2 passed without issues, but then during CamHex warm-up, M2Hex showed oscillations in X, Y, and Z and continued oscillating post CamHex warmup. Brian checked this via the EUI.
We turned off OSS, put LOTO on, for Guillem and Andy to turn on the laser tracker. M1M3 faulted because it was enabled while they entered the cell. After recovering M1M3, we disabled the FA215, Brian turned off the air valve for it. Then we checked the dome moving, telemetry and shutter. After raising M1M3 with FA215 disabled, after that we went through TMA checkout. 
We opened the mirror cover, ran BLOCK-T88 alignment test, turned off the laser tracker. We closed the mirror cover again for the dome shutter opening. We also kept the HVAC setpoint 10°C, turned off HVAC, turn the  VEC-04 on, opened the dome shutter at 20:00h CLT, watching the temperature. We opened the mirror cover, and made dome to follow.
We moved TMA and dome to be 60 deg away from the sun. We tried to take sample image before the twilight flat script, then Camhexapod faulted, due to u band offset not applied to the hexapod. We tried to take images without recovering hexapod, but it faulted MTMount and MTPtg. We updated the hexapod configuration, recover all, took some flats and darks. It was too late to run the BLOCK-215. Then we applied the best offset from last night, with set_dof and ran T235. 
We took some images in between and we found that we were pointing close to the Moon (images 34, 35, 36, 37), we did a 30-degree shift to avoid interference and proceeded with T249.
Applying bending mode 1 and 3 to M2, ran T270 with recycling MTAOS to compare the +-90 degrees. T273 is same test as T270 but for the M1M3. Also during this time Brian adjusted the azimuth limit to 220°.
Handing over to late night shift...
T227


--------- AOS Summary ---------

To answer the big question—no, we didn't break the image quality record tonight, as poor seeing prevailed, with SOAR measurements reaching around 1.85 arcsec. Despite the challenging conditions, we successfully resolved the rotation issue identified yesterday with the bending modes and closed the loop for the first time using all 20 M2 bending modes. We also managed to close the loop with M1M3, using its first 9 modes. We then took at some NGC 55 in different bands, which would be interesting to reduce if possible tomorrow.

When we attempted all 20 modes for M1M3, we encountered force clipping due to differing stress limits between MTAOS and M1M3. We started attempting to fix the tilts and hexapod misalignments, but midway through, we closed the dome due to temperature shifts. Rapid analysis fell over during the night, due to images being passed to it by standardscripts before they were ingested. The issue was resolved, but RA kept slow for the rest of he night. We later reopened, giving us a brief 45 minutes on sky which were dedicated to science validation.
```

Telescope Status: MTM1M3 ENABLED - lowered and enabled to prevent to go to fault
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator needs to be back at 0º (when parking at horizon), in STANDBY w/ cabinets and PXIs powered off
CCCamera, CCOODS, CCHeaderService in ENABLED

*Authors: HyeYun Park, Gonzalo Aravena, Guillem Megias Homar, Brian Stalder, Andrew Connolly, Carlos Morales, Kate Napier*

```
We started around 12:00h CLT powering up the rotator and hexapods. Then we did the M1M3 HP breakaway test passing all HPs. We ran this from EUI to run all HPs at the same time. We ran bump test with the script, and 5 FAs failed (215, 227, 235, 409's primaries and 330 secondary). M1M3 faulted when we put it to disabled state to switch to EUI mode to run the individual FA bump test, it is known issue. OBS-652. We waited air pressure to be lowered to Enable back. Meanwhile we ran hexapod warmups, and comcam usdf checkout. 
We re-ran the individual bump test, FA 330 passed, 227, 235, 409 seemed to be within the range, but Petr identified FA 215 as bad and he asked to close its valve from the M1M3 cell, Brian helped us to performed this action in the cell.
During the hexapod warmups, M2 passed without issues, but then during CamHex warm-up, M2Hex showed oscillations in X, Y, and Z and continued oscillating post CamHex warmup. Brian checked this via the EUI.
We turned off OSS, put LOTO on, for Guillem and Andy to turn on the laser tracker. M1M3 faulted because it was enabled while they entered the cell. After recovering M1M3, we disabled the FA215, Brian turned off the air valve for it. Then we checked the dome moving, telemetry and shutter. After raising M1M3 with FA215 disabled, after that we went through TMA checkout. 
We opened the mirror cover, ran BLOCK-T88 alignment test, turned off the laser tracker. We closed the mirror cover again for the dome shutter opening. We also kept the HVAC setpoint 10°C, turned off HVAC, turn the  VEC-04 on, opened the dome shutter at 20:00h CLT, watching the temperature. We opened the mirror cover, and made dome to follow.
We moved TMA and dome to be 60 deg away from the sun. We tried to take sample image before the twilight flat script, then Camhexapod faulted, due to u band offset not applied to the hexapod. We tried to take images without recovering hexapod, but it faulted MTMount and MTPtg. We updated the hexapod configuration, recover all, took some flats and darks. It was too late to run the BLOCK-215. Then we applied the best offset from last night, with set_dof and ran T235. 
We took some images in between and we found that we were pointing close to the Moon (images 34, 35, 36, 37), we did a 30-degree shift to avoid interference and proceeded with T249.
Applying bending mode 1 and 3 to M2, ran T270 with recycling MTAOS to compare the +-90 degrees. T273 is same test as T270 but for the M1M3. Also during this time Brian adjusted the azimuth limit to 220°.
Handing over to late night shift...
T227


Around 1:45am, the dew point temperature was too close to the outside temperature.  The humidity was 65%.  We followed the recommended weather closure limits and decided to close the dome.  We continued to monitor the weather conditions.  Meanwhile we prepared the TMA for the next test which was a series of u-band images.  
After approximately 30 minutes, we evaluated that the temperatures had stabilized and the humidity had significantly dropped.  We reopened the dome and continued to run the blocks until 5am.  We closed the dome and performed BLOCK-T241 and BLOCK-T220.  


--------- AOS Summary ---------

To answer the big question—no, we didn't break the image quality record tonight, as poor seeing prevailed, with SOAR measurements reaching around 1.85 arcsec. Despite the challenging conditions, we successfully resolved the rotation issue identified yesterday with the bending modes and closed the loop for the first time using all 20 M2 bending modes. We also managed to close the loop with M1M3, using its first 9 modes. We then took at some NGC 55 in different bands, which would be interesting to reduce if possible tomorrow.

When we attempted all 20 modes for M1M3, we encountered force clipping due to differing stress limits between MTAOS and M1M3. We started attempting to fix the tilts and hexapod misalignments, but midway through, we closed the dome due to temperature shifts. Rapid analysis fell over during the night, due to images being passed to it by standardscripts before they were ingested. The issue was resolved, but RA kept slow for the rest of he night. We later reopened, giving us a brief 45 minutes on sky which were dedicated to science validation.
```

Telescope Status: MTM1M3 ENABLED - lowered and disabled
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator needs to be back at 0º (when parking at horizon), in STANDBY w/ cabinets and PXIs powered off
CCCamera, CCOODS, CCHeaderService in ENABLED

Mirror lowered
HVAC set point = 10 deg C
FCU Fans still on 

*Authors: HyeYun Park, Gonzalo Aravena, Guillem Megias Homar, Brian Stalder, Andrew Connolly, Carlos Morales, Kate Napier*

```
We started around 12:00h CLT powering up the rotator and hexapods. M1M3 HP breakaway test done passing all HPs. We ran this from EUI to run all HPs at the same time. We ran bump test with the script, and 5 FAs failed (215, 227, 235, 409's primaries and 330 secondary). M1M3 faulted when we put it to disabled state to switch to EUI mode to run the individual FA bump test, it is known issue. OBS-652. We waited air pressure to be lowered to Enable back. Meanwhile we ran hexapod warmups, and comcam usdf checkout. 
We re-ran the individual bump test, FA 330 passed, 227, 235, 409 seemed to be within the range, but Petr identified FA 215 as bad and he asked to close its valve from the M1M3 cell, Brian agreed to performed this action in the cell. During the hexapod warmups, M2 passed without issues, but then during CamHex warm-up, M2Hex showed oscillations in X, Y, and Z and continued oscillating post CamHex warmup. Brian checked this via the EUI. 
After that, we turned off OSS, put LOTO on, for Guillem and Andy to turn on the laser tracker. M1M3 faulted because it was enabled while they entered the cell. After recovering M1M3, we disabled the FA215, Brian turned off the air valve for it. 
We checked the dome moving, telemetry and shutter. After raising M1M3 with FA215 disabled, we went through TMA checkout. 
We opened the mirror cover, ran BLOCK-T88 alignment test, turned off the laser tracker. We closed the mirror cover again for the dome shutter opening. We kept the HVAC setpoint 10°C, turned off HVAC, turn the  VEC-04 on, opened the dome shutter at 20:00h CLT, watching the temperature. We opened the mirror cover, and made dome to follow. We moved TMA and dome to be 60 deg away from the sun. We tried to take sample image before the twilight flat script, then Camhexapod faulted, due to u band offset not applied to the hexapod. We tried to take images without recovering hexapod, but it faulted MTMount and MTPtg. We updated the hexapod configuration, recover all, took some flats and darks. It was too late to run the BLOCK-215.
We applied the best offset from last night, with set_dof and ran T235. Then we took some images in between, found out we were pointing the Moon, we did a 30-degree shift to avoid interference and proceeded with T249.
Applying bending mode 1 and 3 to M2, ran T270 with recycling MTAOS to compare the +-90 degrees. T273 is same test as T270 but for the M1M3. Also during this time Brian adjusted the azimuth limit to 220°.
Handing over to late night shift...
T227
```

Telescope Status: 

*Authors: HyeYun Park, Gonzalo Aravena, Guillem Megias Homar, Brian Stalder, Andrew Connolly, Carlos Morales, Kate Napier*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-09**

,(times in UTC),
Moon Rise,2024-11-09 16:37:10.235,
Moon Set,2024-11-09 05:33:17.186,
Moon Illumination,49%,(% illuminated)
Astronomical Twilight (morning),2024-11-09 08:12:36.896,(-18 degrees)
Astronomical Twilight (evening),2024-11-09 00:40:42.782,(-18 degrees)
Solar Midnight,2024-11-09 04:26:39.852,
Nautical Twilight (morning),2024-11-09 08:44:52.119,(-12 degrees)
Nautical Twilight (evening),2024-11-09 00:08:27.518,(-12 degrees)
Civil Twilight (morning),2024-11-09 09:15:44.702,(-6 degrees)
Civil Twilight (evening),2024-11-08 23:37:34.858,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTCam,LSSTComCam
Total Night,7:31:54,7:31:54
Total Exposure,5:33:42,1:57:57
Slew time(1),0:00:00,0:00:00
Readout time(2),0:51:17,0:11:11
Time loss to fault,NA,NA
Time loss to weather,NA,NA
Idle time,1:58:11,5:33:56
Number of exposures,1338,292
Mean readout time,0.000639,0.000639
Number of slews(1),0,0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241109&max_day_obs=20241110) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- Should we replace zero with empty string?
- TODO: Do not show empty tables. ie no table when "0 total exposures"
- TODO: Add link to exposures detail when A field value (e.g. observation_reason=science) is clicked on.  The detail should include all exposures counted int the total for that field value.

In [13]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(df)

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 292 total exposures**

Observation Type


,good,questionable,junk,unknown,total
acq,0,0,0,105,105
bias,0,0,0,32,32
cwfs,0,0,0,95,95
dark,0,0,0,32,32
science,0,0,0,28,28


...

,good,questionable,junk,unknown,total
bias,0,0,0,380,380
dark,0,0,0,128,128
flat,0,0,0,830,830


Observation Reason


,good,questionable,junk,unknown,total
bias,0,0,0,20,20
block-t233,0,0,0,119,119
dark,0,0,0,16,16
eo_opsim,0,0,0,115,115
flat,0,0,0,1065,1065
test,0,0,0,3,3


Science Program


,good,questionable,junk,unknown,total
BLOCK-T233,0,0,0,119,119
BLOCK-T262,0,0,0,115,115
E748,0,0,0,1057,1057
test,0,0,0,3,3
unknown,0,0,0,44,44


## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [14]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-09 10:57:28.581261** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Retracting the mirror covers before parking at horizon</pre>


- **2024-11-09 10:49:51.806841** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>lower the mirror</pre>


- **2024-11-09 10:48:26.067541** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Completed hexapod shutdown</pre>


- **2024-11-09 10:39:35.374175** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>rotator shutdown</pre>


- **2024-11-09 10:34:13.672391** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Run BLOCK-T241</pre>


...

- **2024-11-09 15:41:00.876927** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Starting HP breakaway test on M1M3 (BLOCK-T145)</pre>


- **2024-11-09 15:35:16.447985** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Enble M1M3</pre>


- **2024-11-09 15:16:52.897987** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Turning on the rotator</pre>


- **2024-11-09 15:15:42.700719** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Powering up Rotator and Hexapods</pre>


- **2024-11-09 13:23:44.532962** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>u-band installed in ComCam (replacing g).  OSS and PS back on.  moving back to zenith.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [15]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-09 12:00:00**.
- min_dayobs='2024-11-09', max_dayobs='2024-11-10'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [16]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,3,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241109&max_day_obs=20241110
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
exposurelog/exposures.LSSTComCam,292,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
exposurelog/exposures.LSSTCam,1338,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110
narrativelog/messages,214,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-09T00%3A00%3A00&max_date_added=2024-11-10T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [17]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [3]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241109&max_day_obs=20241110

None
- Got 3 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [214]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-09T00%3A00%3A00&max_date_added=2024-11-10T00%3A00%3A00

None
- Got 214 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [20]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

<font style="color:lightblue;background-color:None;">No exposurelog records found 2024-11-09 to 2024-11-10. </font>

Used [API Data](https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241109&max_day_obs=20241110)

## Finale

In [21]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-12 01:53:58.149098
